## **네이버 카페 게시글 제목 수집**

* **학습 목표**
    * 네이버 카페 사이트에서 자유게시판 위치를 입력하면 게시글 제목을 크롤링할 수 있다.

    ※ <u>카페마다 자유게시판 위치가 다르므로 input할 수 있도록 parameter를 설정한다.</u><br>
    ※ <u>페이지에 따라 css_selector가 바뀌니 주의해야 한다.</u><br>
    ※ <u>네이버의 경우 게시글 영역은 iframe으로 구성되어 있다. 따라서 창 전환이 필요하다.</u>

<div markdown="1" style="text-align:center; margin-bottom:10px">
<img src="./images/naver_cafe.PNG" width="50%">
</div> 


### **Settings**

In [123]:
# Settings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd 

### **1) 하나씩 살펴보기**

#### **Chrome 창 열기**

In [124]:
# Chrome 창 열기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64)")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 10.6MB/s]


#### **카페 URL 이동**

In [126]:
# 카페 URL로 이동
url = 'https://cafe.naver.com/eleccigar'
driver.get(url)

#### **자유게시판 클릭**

In [127]:
# navigation bar에서 자유게시판 Click
nav_css = '#menuLink15'
driver.find_element(By.CSS_SELECTOR, nav_css).click()

#### **게시글 영역 창 전환**

In [128]:
# 게시글 iframe으로 창 전환
driver.switch_to.frame('cafe_main')
# driver.switch_to.default_content()        # 원래 창으로 돌아감

#### **글 목록 가져오기(해당 페이지만)**

In [132]:
# 게시글 목록 가져오기
## 게시글 목록은 공지, 추천, 글 세가지로 분류되는데 추천글이 있으면 5 아니면 4 이다.
posts_css = '#main-area > div:nth-child(5) > table > tbody > tr'
posts = driver.find_elements(By.CSS_SELECTOR, posts_css)

In [135]:
# 게시글 제목 출력하기
post_lst = []

for post in posts:
    title = post.find_element(By.CSS_SELECTOR, 'td > div.board-list > div.inner_list > a').text
    post_lst.append(title)

post_lst

['아웅 배터지네여',
 '가습이 왔어요',
 '기기 변경 ㅠ',
 '[일상] 폰에 모아둔 생활팁 들이요~',
 '아보카도 새 팟인데 초기불량인가여?',
 '피코 구했습니다ㅋㅋ',
 '[일상] 발맥 추가로 들였는데',
 '[일상] 상여자의 avp ax 코팅',
 '[일상] 큰 맘먹고 rda 주문했는데',
 '[전담뉴스] 아스몬 전색상 재입고됐네요',
 '[일상] 목감기 걸린거 같은데',
 '테일러 마르키사 알로에 곰돌이 젤리 액상',
 'avp ax~~~♡',
 '유행한다는 식빵등 했어요~',
 '[일상] 좀 늦게 알아버린 바보같은....']

#### **글 목록 가져오기(여러 페이지)**

In [145]:
# 게시글 제목 출력하기(여러 페이지)
pages = 12
post_lst = []

page = 1   
while True:
    # 게시글 목록 가져오기
    posts_css = '#main-area > div:nth-child(5) > table > tbody > tr'
    posts = driver.find_elements(By.CSS_SELECTOR, posts_css)

    # 게시글 제목 가져오기
    for post in posts:
        title = post.find_element(By.CSS_SELECTOR, 'td > div.board-list > div.inner_list > a').text
        post_lst.append(title)
    
    # 페이지 조작
    if page == pages:       # 현재 페이지가 pages이면 stop
        break
    elif page % 10 == 0:    # 현재 페이지가 10의 배수이면 '다음' 클릭
        next_css = '#main-area > div.prev-next > a.pgR'
        driver.find_element(By.CSS_SELECTOR, next_css).click()
    else:                   # 다음 페이지 클릭
        pos = 2 if page <= 10 else 3    # 첫 페이지는 이전 버튼이 없어서 2 그 이후로는 3이다.
        page_css = f'#main-area > div.prev-next > a:nth-child({page % 10 + pos})'
        driver.find_element(By.CSS_SELECTOR, page_css).click()

    page += 1

post_lst

['아웅 배터지네여',
 '가습이 왔어요',
 '기기 변경 ㅠ',
 '[일상] 폰에 모아둔 생활팁 들이요~',
 '아보카도 새 팟인데 초기불량인가여?',
 '피코 구했습니다ㅋㅋ',
 '[일상] 발맥 추가로 들였는데',
 '[일상] 상여자의 avp ax 코팅',
 '[일상] 큰 맘먹고 rda 주문했는데',
 '[전담뉴스] 아스몬 전색상 재입고됐네요',
 '[일상] 목감기 걸린거 같은데',
 '테일러 마르키사 알로에 곰돌이 젤리 액상',
 'avp ax~~~♡',
 '유행한다는 식빵등 했어요~',
 '[일상] 좀 늦게 알아버린 바보같은....',
 '[감동] 감사합니다 팍스베이프ㅜㅜ',
 '[일상] 액상이 왔네요.. ㅎㅎㅎ',
 '울적한 하루',
 '전담 추천해주세요',
 '[일상] 아주재밌는걸 발견했네요',
 '이노포그 만족하면서 사용하고 있는데...질문이요',
 '피코 브실이 너무땡기네용...ㅠ',
 '[일상] 유물 발견',
 '[일상] 기추 했습니다',
 '발라 맥스 질문',
 '하....ㅠㅡㅠ',
 '[일상] avp ax 색상',
 '[일상] 다크가 왔어요',
 '기기 주문한거마냥 AVP AX 겁나 빨리 오고 있네요 ^^',
 '리즌 18350 개이뿌네 ㅠㅠ',
 '아ㅏㅏㅏ....... 술먹고 전담 잃어버렸네용 ㅠㅠㅠㅠ',
 '이번 베이프 엑스포는,',
 '[일상] 섞먹도 매력있네요 ㅎㅎ',
 '핸첵 말머리 있는 줄 모르고 하나하나 치고 있었네요ㅋㅋㅋㅋ',
 '둘 중 하나를 나눔한다면 무엇이 끌리시나요..?',
 '[일상] 이시간 즐거운 벙개~^^',
 '좋은건 7개👍🏻럭키 쎄븐😉✨',
 '[일상] 브이쓰루 기추했습니다~',
 '젤로로 입문, 쓰루로 기추...',
 '한달 5만원으로 전담 즐기는 여러가지 방법 공유해 봐요 ㅋㅋㅋ',
 '[일상] 오늘 하루 길고 기네요..',
 '오늘도 한가하네여 ㅎㅎ',
 '전담상회 추천인 한분 해드려영~',
 '[일상] 메이데이 자허블',
 '[일상] 또 충동구매했네요',
 '젤로 와트수는 원래 계속 바뀌나요? ㅜㅜ',


### **2) 함수로 나타내기**

In [169]:
def naver_cafe_title_crawling(url, nav_css, recommend, pages):
    prt = 10                          # 최대 출력 수
    interval = (pages // prt) + 1     # 출력할 페이지 간격

    # 카페 URL로 이동
    driver.get(url)         

    # navigation bar에서 자유게시판 Click
    driver.find_element(By.CSS_SELECTOR, nav_css).click()
    
    # 게시글 iframe으로 창 전환
    driver.switch_to.frame('cafe_main')

    post_lst = []

    depth = 5 if recommend == 1 else 4
    page = 1   
    
    print('Start!', end='')
    while True:
        if page % interval == 0:
            print(f' > {page}', end='')
        
        # 게시글 목록 가져오기
        posts_css = f'#main-area > div:nth-child({depth}) > table > tbody > tr'
        posts = driver.find_elements(By.CSS_SELECTOR, posts_css)

        # 게시글 제목 가져오기
        for post in posts:
            title = post.find_element(By.CSS_SELECTOR, 'td > div.board-list > div.inner_list > a').text
            # post_lst.append(title)    # title만 저장
            post_lst.append([url.split('/')[-1], title])
        
        # 페이지 조작
        if page == pages:       # 현재 페이지가 pages이면 stop
            print(' >>> End!')
            break
        elif page % 10 == 0:    # 현재 페이지가 10의 배수이면 '다음' 클릭
            next_css = '#main-area > div.prev-next > a.pgR'
            driver.find_element(By.CSS_SELECTOR, next_css).click()
        else:                   # 다음 페이지 클릭
            pos = 2 if page <= 10 else 3    # 첫 페이지는 이전 버튼이 없어서 2 그 이후로는 3이다.
            page_css = f'#main-area > div.prev-next > a:nth-child({page % 10 + pos})'
            driver.find_element(By.CSS_SELECTOR, page_css).click()

        page += 1
    
    return post_lst


### **3) 실습**

In [ ]:
# Chrome 창 열기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64)")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

#### **전자담배 카페**

In [170]:
# 전자담배 카페

url = 'https://cafe.naver.com/eleccigar'
nav_css = '#menuLink15'
recommend = 1
pages = 12

posts = naver_cafe_title_crawling(url, nav_css, recommend, pages)

Start! > 2 > 4 > 6 > 8 > 10 > 12 >>> End!


#### **이찬원 글로벌 팬카페**

In [171]:
# 이찬원 글로벌 팬카페

url = 'https://cafe.naver.com/wordglobalchans'
nav_css = '#menuLink11'
recommend = 0
pages = 12

posts = naver_cafe_title_crawling(url, nav_css, recommend, pages)

Start! > 2 > 4 > 6 > 8 > 10 > 12 >>> End!


#### **하나의 데이터프레임으로 통합**

In [172]:
# 여러 카페의 게시물 하나의 데이터 프레임으로 통합
pages = 12
cafe_dict = {'https://cafe.naver.com/eleccigar':('#menuLink15', 1),
             'https://cafe.naver.com/wordglobalchans':('#menuLink11', 0)}

cafe_posts = []
for url, (nav_css, recommend) in cafe_dict.items():
    posts = naver_cafe_title_crawling(url, nav_css, recommend, pages)
    cafe_posts.extend(posts)

Start! > 2 > 4 > 6 > 8 > 10 > 12 >>> End!
Start! > 2 > 4 > 6 > 8 > 10 > 12 >>> End!


In [173]:
df = pd.DataFrame(cafe_posts, columns=['cafe','post'])
df    

,cafe,post
0,eleccigar,아웅 배터지네여
1,eleccigar,가습이 왔어요
2,eleccigar,기기 변경 ㅠ
3,eleccigar,[일상] 폰에 모아둔 생활팁 들이요~
4,eleccigar,아보카도 새 팟인데 초기불량인가여?
...,...,...
355,wordglobalchans,世界一好きな歌手イチャンウォン
356,wordglobalchans,이쁜 이찬원 홧팅
357,wordglobalchans,전세계 찬스님들 모여주세요
358,wordglobalchans,이찬원 므싯다 매력남 최고
